In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [3]:
# for Colab
from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/train_set.csv')
df['note'] = df['note'].apply(lambda x: int(x))

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('train_set.csv',index_col=0)
df['note'] = df['note'].apply(lambda x: int(x))
df.head(2)

,note,avis,assureur,produit,date_publication,date_exp,avis_en,tokens_en,bigrams_en,tokens_fr,bigrams_fr
0,4,la personne au téléphone était clair et sympat...,L'olivier Assurance,auto,06/10/2021,01/10/2021,the person on the phone was clear and friendly...,"['person', 'phone', 'clear', 'friendly', 'expl...","['person_phone', 'phone_clear', 'clear_friendl...","['personne', 'téléphone', 'clair', 'sympathiqu...","['personne_téléphone', 'téléphone_clair', 'cla..."
1,4,satisfaitréactivité simplicité prix attractif ...,APRIL Moto,moto,09/07/2021,01/07/2021,satisfiedreactivity simplicity attractive pric...,"['satisfiedreactivity', 'simplicity', 'attract...","['satisfiedreactivity_simplicity', 'simplicity...","['satisfaitréactivité', 'simplicité', 'prix', ...","['satisfaitréactivité_simplicité', 'simplicité..."


### Différents preprocessing

In [3]:
# Preprocessing the text with lemmatization
def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    words_list = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(words_list)

In [ ]:
# Preprocessing the text with stemming
def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(stemmed_tokens)

In [ ]:
# Preprocessing the text without stemming and lemmatizing
def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    toks = [word for word in tokens if word not in stop_words]
    return ' '.join(toks)

## TF_IDF

### SVM

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df['avis_en'], df['note'], test_size=0.2, random_state=42)
# Preprocessing
X_train_pre = X_train.apply(preprocess)
X_test_pre = X_test.apply(preprocess)
# TF IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train_pre)
X_test_tfidf = vectorizer.transform(X_test_pre)
# Training model
from sklearn.svm import SVC
model = SVC(class_weight='balanced')
model.fit(X_train_tfidf, y_train)
predictions = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, predictions))

Accuracy: 0.5098527276498652
              precision    recall  f1-score   support

           1       0.64      0.74      0.68      1444
           2       0.36      0.37      0.37       716
           3       0.32      0.22      0.26       665
           4       0.45      0.41      0.43       999
           5       0.54      0.58      0.56       997

    accuracy                           0.51      4821
   macro avg       0.46      0.46      0.46      4821
weighted avg       0.49      0.51      0.50      4821



In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df['avis_en'], df['note'], test_size=0.2, random_state=42)
# Preprocessing
X_train_pre = X_train.apply(preprocess)
X_test_pre = X_test.apply(preprocess)
# Adding bigrams to TF IDF
vectorizer = TfidfVectorizer(ngram_range=(2,2))
X_train_tfidf = vectorizer.fit_transform(X_train_pre)
X_test_tfidf = vectorizer.transform(X_test_pre)
# Entrainement modèle
from sklearn.svm import SVC
model = SVC(class_weight='balanced')
model.fit(X_train_tfidf, y_train)
predictions = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, predictions))

Accuracy: 0.4662933001451981
              precision    recall  f1-score   support

           1       0.47      0.94      0.62      1444
           2       0.34      0.08      0.13       716
           3       0.39      0.05      0.09       665
           4       0.45      0.33      0.38       999
           5       0.51      0.48      0.49       997

    accuracy                           0.47      4821
   macro avg       0.43      0.37      0.34      4821
weighted avg       0.44      0.47      0.40      4821



## LTSM

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Reshape

#### V1

In [ ]:
# Sampling the data
sample_df = df.sample(n=8000, random_state=42)
# Separate the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    sample_df['avis_en'],
    sample_df['note'],
    test_size=0.2,
    random_state=42)

# Apply preprocessing
X_train_processed= X_train.apply(preprocess)
X_test_processed = X_test.apply(preprocess)
# Convert X_train and X_test to USE embeddings
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X_train_use = np.array(use(X_train_processed.tolist()))
X_test_use = np.array(use(X_test_processed.tolist()))
# Adjust labels to start from 0
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

# LSTM model with USE embeddings
model = Sequential()
model.add(Reshape((1, X_train_use.shape[1]), input_shape=(X_train_use.shape[1],)))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

# Compile and training of the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_use, y_train_adjusted, validation_split=0.2, epochs=100, batch_size=200, verbose=1)
# Evaluation of the model
accuracy = model.evaluate(X_test_use, y_test_adjusted)[1]
print(f'Accuracy: {accuracy}')

Epoch 1/100
26/26 [==============================] - 3s 27ms/step - loss: 1.5824 - accuracy: 0.3336 - val_loss: 1.5383 - val_accuracy: 0.3500
Epoch 2/100
26/26 [==============================] - 0s 9ms/step - loss: 1.4802 - accuracy: 0.4082 - val_loss: 1.4027 - val_accuracy: 0.4281
Epoch 3/100
26/26 [==============================] - 0s 10ms/step - loss: 1.3362 - accuracy: 0.4656 - val_loss: 1.2698 - val_accuracy: 0.4727
Epoch 4/100
26/26 [==============================] - 0s 10ms/step - loss: 1.2298 - accuracy: 0.4852 - val_loss: 1.2069 - val_accuracy: 0.4789
Epoch 5/100
26/26 [==============================] - 0s 10ms/step - loss: 1.1804 - accuracy: 0.4986 - val_loss: 1.1865 - val_accuracy: 0.4852
Epoch 6/100
26/26 [==============================] - 0s 15ms/step - loss: 1.1561 - accuracy: 0.5021 - val_loss: 1.1766 - val_accuracy: 0.4844
Epoch 7/100
26/26 [==============================] - 0s 12ms/step - loss: 1.1403 - accuracy: 0.5092 - val_loss: 1.1724 - val_accuracy: 0.4852
Epoch 8

#### V2

In [ ]:
# Sampling the data
sample_df = df.sample(n=8000, random_state=42)
# Separate the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    sample_df['avis_en'],
    sample_df['note'],
    test_size=0.2,
    random_state=42)

# Apply preprocessing
X_train_processed= X_train.apply(preprocess)
X_test_processed = X_test.apply(preprocess)
# Convert X_train and X_test to USE embeddings
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X_train_use = np.array(use(X_train_processed.tolist()))
X_test_use = np.array(use(X_test_processed.tolist()))
# Adjust labels to start from 0
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=15)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.0001)

# LSTM model with USE embeddings
model = Sequential()
model.add(Reshape((1, X_train_use.shape[1]), input_shape=(X_train_use.shape[1],)))
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train_use, y_train_adjusted, validation_split=0.2, epochs=200, batch_size=100, verbose=1, callbacks=[es, mc, lr])
# Evaluate the model on the test set
accuracy = model.evaluate(X_test_use, y_test_adjusted)[1]
print(f'Accuracy: {accuracy}')

Epoch 1/200



44/52 [========================>.....] - ETA: 0s - loss: 1.5731 - accuracy: 0.3636
Epoch 1: val_accuracy improved from -inf to 0.42500, saving model to best_model.h5
52/52 [==============================] - 4s 26ms/step - loss: 1.5635 - accuracy: 0.3719 - val_loss: 1.4868 - val_accuracy: 0.4250 - lr: 0.0010
Epoch 2/200
 9/52 [====>.........................] - ETA: 0s - loss: 1.4827 - accuracy: 0.4422

c:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/52 [===========================>..] - ETA: 0s - loss: 1.3870 - accuracy: 0.4528
Epoch 2: val_accuracy improved from 0.42500 to 0.45937, saving model to best_model.h5
52/52 [==============================] - 1s 10ms/step - loss: 1.3847 - accuracy: 0.4525 - val_loss: 1.2726 - val_accuracy: 0.4594 - lr: 0.0010
Epoch 3/200
45/52 [========================>.....] - ETA: 0s - loss: 1.2536 - accuracy: 0.4593
Epoch 3: val_accuracy improved from 0.45937 to 0.48203, saving model to best_model.h5
52/52 [==============================] - 0s 9ms/step - loss: 1.2484 - accuracy: 0.4613 - val_loss: 1.2016 - val_accuracy: 0.4820 - lr: 0.0010
Epoch 4/200
49/52 [===========================>..] - ETA: 0s - loss: 1.2050 - accuracy: 0.4802
Epoch 4: val_accuracy did not improve from 0.48203
52/52 [==============================] - 0s 9ms/step - loss: 1.2031 - accuracy: 0.4809 - val_loss: 1.1806 - val_accuracy: 0.4773 - lr: 0.0010
Epoch 5/200
45/52 [========================>.....] - ETA: 0s - loss: 1.1853 -

### V3

In [ ]:
# Separate the data into training and testing sets
sample_df = df.sample(n=8000, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    sample_df['avis_en'],
    sample_df['note'],
    test_size=0.2,
    random_state=42
)

# Apply preprocessing to the training and testing data
X_train_processed = X_train.apply(preprocess)
X_test_processed = X_test.apply(preprocess)

# Load Universal Sentence Encoder
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Convert text to USE embeddings
X_train_use = use(X_train_processed.tolist())
X_test_use = use(X_test_processed.tolist())

y_train_adjusted = y_train-1
y_test_adjusted = y_test-1

es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=15)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.0001)

from tensorflow.keras.layers import Input, Reshape, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# LSTM model with USE embeddings
model = Sequential()
model.add(Input(shape=(X_train_use.shape[1],)))
model.add(Reshape((1, X_train_use.shape[1])))
model.add(LSTM(64, return_sequences=True, activation='relu'))  # another LSTM layer
model.add(Dropout(0.5))
model.add(LSTM(32, activation='relu'))  # final LSTM layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # Output layer with 5 neurons

optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train_use, y_train_adjusted, validation_split=0.2, epochs=200, batch_size=100, verbose=1, callbacks=[es, mc, lr])
# Evaluate the model on the test set
accuracy = model.evaluate(X_test_use, y_test_adjusted)[1]
print(f'Accuracy: {accuracy}')

Epoch 1/200
47/52 [==========================>...] - ETA: 0s - loss: 1.5917 - accuracy: 0.3013
Epoch 1: val_accuracy improved from -inf to 0.28906, saving model to best_model.h5
52/52 [==============================] - 5s 24ms/step - loss: 1.5903 - accuracy: 0.3018 - val_loss: 1.5633 - val_accuracy: 0.2891 - lr: 0.0010
Epoch 2/200
49/52 [===========================>..] - ETA: 0s - loss: 1.4750 - accuracy: 0.3661
Epoch 2: val_accuracy improved from 0.28906 to 0.45703, saving model to best_model.h5
52/52 [==============================] - 1s 11ms/step - loss: 1.4710 - accuracy: 0.3717 - val_loss: 1.2701 - val_accuracy: 0.4570 - lr: 0.0010
Epoch 3/200
47/52 [==========================>...] - ETA: 0s - loss: 1.2943 - accuracy: 0.4572
Epoch 3: val_accuracy improved from 0.45703 to 0.46562, saving model to best_model.h5
52/52 [==============================] - 1s 11ms/step - loss: 1.2915 - accuracy: 0.4596 - val_loss: 1.1946 - val_accuracy: 0.4656 - lr: 0.0010
Epoch 4/200
48/52 [============

## Bert Hugging face

In [4]:
from transformers import pipeline
# Load the sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Example usage
review = df["avis_en"].iloc[3]
result = sentiment_pipeline(review)
result

[{'label': '2 stars', 'score': 0.341024249792099}]

In [12]:
sample_df = df.sample(n=8000, random_state=42)
# Target transformation
sample_df['target'] = sample_df['note'].apply(lambda x: f"{x} stars")
# Preprocessing
avis_processed = sample_df["avis_en"].apply(preprocess)
# Truncation
avis_truncated = avis_processed.apply(lambda x: x[:512])
# Predicting the rating of the reviews
preds = avis_truncated.apply(lambda x: sentiment_pipeline(x)[0]['label'])

In [14]:
# Compute accuracy manually
correct_predictions = (sample_df["target"] == preds).sum()
total_predictions = len(sample_df)
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy}")

Accuracy: 0.217125
